# **SIT731**
**Name: Addala Srivatsa Gaurav**

**Student ID: S223872808**

**Email id: asgaurav9@gmail.com**

**I am Post Graduate Student(SIT731)**

# Introduction
This project delves into the nycflights13 dataset, capturing details of 336,776 flights departing from New York's EWR, JFK, and LGA airports to destinations and nearby regions in 2013. Utilizing pandas, we establish a connection with an SQLite database and replicate SQL queries for comprehensive data analysis. Five crucial datasets, including flight information, carrier details, airport data, plane specifics, and meteorological records, are incorporated. The analysis is presented in a Jupyter/IPython notebook using jupytext, combining a .py source file with a .ipynb notebook for effective code interpretation. The objective is to showcase pandas' capabilities in data exploration and manipulation.

In [ ]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option("display.notebook_repr_html", False)

# Load data from CSV files
airlines = pd.read_csv("airlines.csv", comment='#')
airports = pd.read_csv("airports.csv", comment="#")
flights = pd.read_csv("flights.csv", comment="#")
planes = pd.read_csv("planes.csv", comment="#")
weather = pd.read_csv("weather.csv", comment="#")

This code utilizes pandas to load data from CSV files ('airlines.csv', 'airports.csv', 'flights.csv', 'planes.csv', 'weather.csv'). It sets display options and prints the first few rows of each DataFrame (airlines, airports, flights, planes, weather) for initial exploration, providing an overview of the dataset structure and content. The comment='#' parameter excludes lines starting with '#' in the CSV files during loading. The code also imports necessary libraries such as sqlite3, numpy, matplotlib, and seaborn for further analysis and visualization.

In [ ]:
import tempfile, os.path
dbfile = os.path.join(tempfile.mkdtemp(), "nyc.db")
print(dbfile)

In [ ]:
conn = sqlite3.connect(dbfile)

In [ ]:
airlines.to_sql("airlines", conn, index=False)
airports.to_sql("airports", conn, index=False)
flights.to_sql("flights", conn, index=False)
planes.to_sql("planes", conn, index=False)
weather.to_sql("weather", conn, index=False)

In [ ]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

This Python code creates a temporary SQLite database file ('nyc.db') using tempfile and os.path. It establishes a connection to the database using sqlite3. Data from DataFrames (airlines, airports, flights, planes, weather) is then stored in corresponding tables in the SQLite database. The tables' names are queried and printed to verify successful creation.

In [ ]:
# Pandas Equivalent
task1_my = planes['engine'].drop_duplicates().reset_index(drop=True).to_frame()

# SQL Query
task1_sql = pd.read_sql_query("""
    SELECT DISTINCT engine FROM planes
""", conn)

# Check for equality
pd.testing.assert_frame_equal(task1_sql, task1_my)

This code shows the equivalent Pandas implementation of a SQL query selecting distinct values from the 'engine' column in the 'planes' table. The results from both SQL and Pandas are compared using pd.testing.assert_frame_equal. If there is no error, it means the results match. The SQL and Pandas results are then printed for verification.

In [ ]:
# Pandas Code:
task2_my = planes[['type', 'engine']].drop_duplicates().reset_index(drop=True)

# SQL Code:
task2_sql = pd.read_sql_query("""
    SELECT DISTINCT type, engine FROM planes
""", conn)

# Check for equality
pd.testing.assert_frame_equal(task2_sql, task2_my)

This code demonstrates the equivalent Pandas implementation of a SQL query. The goal is to select distinct combinations of values from the 'type' and 'engine' columns in the 'planes' table. The results from both SQL and Pandas are compared using pd.testing.assert_frame_equal. If there is no error, it means the results match. The SQL and Pandas results are then printed for verification.

In [ ]:
# SQL Query:
task3_sql = pd.read_sql_query("""
    SELECT COUNT(*), engine
    FROM planes
    GROUP BY engine
""", conn)

# Pandas Equivalent
task3_my = planes.groupby('engine').size().reset_index(name='count').sort_values('engine')

# Reorder the columns to match the SQL result
task3_my = task3_my[['count', 'engine']]

# Compare SQL and Pandas results 
try:
    pd.testing.assert_frame_equal(task3_sql, task3_my, check_dtype=False)
except AssertionError as e:
    print(f"\nAssertionError: {e}")

This code represents an SQL query and its Pandas equivalent. Both aim to count the occurrences of each unique 'engine' value in the 'planes' table. The SQL and Pandas results are then compared using pd.testing.assert_frame_equal. The check_dtype=False parameter is used to ignore data type differences that might arise from certain operations. If there is no error, it indicates that the results are equivalent. Finally, the SQL and Pandas results are printed for verification

In [ ]:
# SQL Query
task4_sql = pd.read_sql_query("""
    SELECT COUNT(*) AS count, engine, type FROM planes GROUP BY engine, type
""", conn)

# Pandas Equivalent
task4_pandas = planes.groupby(['engine', 'type']).size().reset_index(name='count')[['count', 'engine', 'type']]


# Compare the results
pd.testing.assert_frame_equal(task4_sql, task4_pandas)

This code consists of an SQL query and its Pandas equivalent. Both queries aim to count the occurrences of unique combinations of 'engine' and 'type' values in the 'planes' table. The SQL and Pandas results are then compared using pd.testing.assert_frame_equal. If there is no error, it indicates that the results are equivalent. Finally, the SQL and Pandas results are printed for verification.

In [ ]:
# SQL Query
task5_sql = pd.read_sql_query("""
    SELECT MIN(year) AS min_year, AVG(year) AS avg_year, MAX(year) AS max_year, engine, manufacturer
    FROM planes
    GROUP BY engine, manufacturer
""", conn)


# Pandas equivalent code for the SQL query
task5_pandas = planes.groupby(['engine', 'manufacturer']).agg({
    'year': ['min', 'mean', 'max']
}).reset_index()

# Flatten the multi-level columns and rename
task5_pandas.columns = ['engine', 'manufacturer', 'min_year', 'avg_year', 'max_year']

# Reorder columns to match SQL result
task5_pandas = task5_pandas[['min_year', 'avg_year', 'max_year', 'engine', 'manufacturer']]



pd.testing.assert_frame_equal(task5_sql, task5_pandas, check_like=True)

The SQL query retrieves the minimum, average, and maximum 'year' values grouped by 'engine' and 'manufacturer' from the 'planes' table. The Pandas equivalent achieves the same using the groupby and agg functions, creating a DataFrame with consistent column names. The results are compared using pd.testing.assert_frame_equal. If no error is raised, the Pandas implementation is consistent with the SQL query. The DataFrames are printed for verification.


SQL Query
task6_sql = pd.read_sql_query("""
    SELECT * FROM planes WHERE speed IS NOT NULL
""", conn)

Pandas equivalent code for the SQL query
task6_pandas = planes[planes['speed'].notnull()]

task6_pandas = task6_pandas.reset_index(drop=True)


Compare the DataFrames
pd.testing.assert_frame_equal(task6_sql, task6_pandas)

The SQL query retrieves all records from the 'planes' table where the 'speed' column is not null. The Pandas equivalent filters the DataFrame using boolean indexing. The DataFrames are compared using pd.testing.assert_frame_equal. If no error is raised, the Pandas implementation is consistent with the SQL query. Results for both SQL and Pandas are printed for verification.

In [ ]:
# SQL Query
task7_sql = pd.read_sql_query("""
    SELECT tailnum FROM planes
    WHERE seats BETWEEN 150 AND 210 AND year >= 2011
""", conn)

# Pandas equivalent code for the SQL query
task7_pandas = planes[(planes['seats'].between(150, 210)) & (planes['year'] >= 2011)][['tailnum']]

task7_pandas = task7_pandas.reset_index(drop=True)



pd.testing.assert_frame_equal(task7_sql, task7_pandas)

The SQL query retrieves 'tailnum' values from the 'planes' table where 'seats' fall between 150 and 210, and 'year' is greater than or equal to 2011. The Pandas equivalent uses boolean indexing to filter the DataFrame accordingly. Both results are compared using pd.testing.assert_frame_equal for consistency.

SQL Query
task8_sql = pd.read_sql_query("""
    SELECT tailnum, manufacturer, seats FROM planes
    WHERE manufacturer IN ("BOEING", "AIRBUS", "EMBRAER") AND seats > 390
""", conn)


Pandas equivalent code for the SQL query
manufacturers_list = ["BOEING", "AIRBUS", "EMBRAER"]
task8_pandas = planes[(planes['manufacturer'].isin(manufacturers_list)) & (planes['seats'] > 390)][['tailnum', 'manufacturer', 'seats']]

task8_pandas = task8_pandas.reset_index(drop=True)



pd.testing.assert_frame_equal(task8_sql, task8_pandas)

The SQL query extracts 'tailnum,' 'manufacturer,' and 'seats' from the 'planes' table where 'manufacturer' is in a specified list ("BOEING," "AIRBUS," "EMBRAER") and 'seats' exceed 390. The Pandas equivalent utilizes boolean indexing with isin and logical conditions to filter the DataFrame accordingly. The results are compared using pd.testing.assert_frame_equal to ensure consistency.

In [ ]:
# SQL Query
task9_sql = pd.read_sql_query("""
    SELECT DISTINCT year, seats FROM planes
    WHERE year >= 2012
    ORDER BY year ASC, seats DESC
""", conn)

# Pandas equivalent code for the SQL query
task9_pandas = planes[planes['year'] >= 2012][['year', 'seats']].sort_values(by=['year', 'seats'], ascending=[True, False]).drop_duplicates()

task9_pandas = task9_pandas.reset_index(drop=True)



pd.testing.assert_frame_equal(task9_sql, task9_pandas)

The SQL query selects distinct 'year' and 'seats' from the 'planes' table where 'year' is greater than or equal to 2012. It orders the results by 'year' in ascending order and 'seats' in descending order. The Pandas equivalent uses boolean indexing, column selection, sorting, and dropping duplicates to achieve the same result. The DataFrames are then compared using pd.testing.assert_frame_equal for consistency.

In [ ]:
# SQL Query
task10_sql = pd.read_sql_query("""
    SELECT DISTINCT year, seats FROM planes
    WHERE year >= 2012
    ORDER BY seats DESC, year ASC
""", conn)


# Pandas equivalent code for the SQL query
task10_pandas = planes[planes['year'] >= 2012][['year', 'seats']].sort_values(by=['seats', 'year'], ascending=[False, True]).drop_duplicates()

task10_pandas = task10_pandas.reset_index(drop=True)



pd.testing.assert_frame_equal(task10_sql, task10_pandas)

The SQL query selects distinct 'year' and 'seats' from the 'planes' table where 'year' is greater than or equal to 2012. It orders the results by 'seats' in descending order and 'year' in ascending order. The Pandas equivalent achieves the same result using boolean indexing, column selection, sorting, and dropping duplicates. The DataFrames are compared for consistency.

In [ ]:
# SQL Query
task11_sql = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) FROM planes
    WHERE seats > 200
    GROUP BY manufacturer
""", conn)


# Pandas equivalent code for the SQL query
task11_pandas = planes[planes['seats'] > 200].groupby('manufacturer').size().reset_index(name='COUNT(*)')

task11_pandas = task11_pandas.sort_values(by=['manufacturer']).reset_index(drop=True)




pd.testing.assert_frame_equal(task11_sql, task11_pandas)

The SQL query retrieves the count of planes with more than 200 seats for each manufacturer from the 'planes' table. The Pandas equivalent filters the DataFrame based on seats, groups by the manufacturer, calculates the size, and resets the index for consistency. Both results are compared using assert_frame_equal to ensure consistency.

In [ ]:
# SQL Query
task12_sql = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) FROM planes
    GROUP BY manufacturer
    HAVING COUNT(*) > 10
""", conn)

# Pandas equivalent code for the SQL query
manufacturer_counts = planes.groupby('manufacturer').size().reset_index(name='COUNT(*)')
task12_pandas = manufacturer_counts[manufacturer_counts['COUNT(*)'] > 10]

task12_pandas = task12_pandas.sort_values(by=['manufacturer']).reset_index(drop=True)



pd.testing.assert_frame_equal(task12_sql, task12_pandas)

The SQL query retrieves the count of planes for each manufacturer from the 'planes' table and filters only those with counts greater than 10 using the having clause. The Pandas equivalent code calculates the manufacturer counts, filters based on the count condition, and resets the index for consistency. Both results are compared using assert_frame_equal to ensure consistency.

In [ ]:
# SQL Query for Task 13 with HAVING clause
task13_sql= pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) AS count
    FROM planes
    WHERE seats > 200
    GROUP BY manufacturer
    HAVING COUNT(*) > 10;
""", conn)

# Pandas Equivalent for Task 13 with HAVING clause
task13_pandas= planes[planes['seats'] > 200].groupby('manufacturer').size().reset_index(name='count')
task13_pandas= task13_pandas[task13_pandas['count'] > 10].reset_index(drop=True)




pd.testing.assert_frame_equal(task13_sql, task13_pandas)

Task 13 SQL query filters planes with more than 200 seats, groups them by manufacturer, and retrieves the count for each group, applying a HAVING clause to select only those with counts exceeding 10. The Pandas equivalent code follows a similar approach, calculating manufacturer counts, filtering based on the count condition, and ensuring consistent indexing. The results are printed and compared using assert_frame_equal to validate equivalence.

In [ ]:
# SQL Query for Task 14
task14_sql = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) AS howmany
    FROM planes
    GROUP BY manufacturer
    ORDER BY howmany DESC
    LIMIT 10;
""", conn)

# Pandas Equivalent 
task14_pandas = planes.groupby('manufacturer').size().reset_index(name='howmany')
task14_pandas = task14_pandas.sort_values(by='howmany', ascending=False).head(10).reset_index(drop=True)





pd.testing.assert_frame_equal(task14_sql, task14_pandas)


Task 14 SQL query counts the number of planes for each manufacturer, orders the result by count in descending order, and limits the output to the top 10 manufacturers. The Pandas equivalent code utilizes groupby and size to obtain manufacturer counts, sorts in descending order, and selects the top 10. SQL and Pandas results are printed and compared using assert_frame_equal to ensure consistency.

In [ ]:
# SQL Query for Task 15
task15_sql = pd.read_sql_query("""
    SELECT
        flights.*,
        planes.year AS plane_year,
        planes.speed AS plane_speed,
        planes.seats AS plane_seats
    FROM flights
    LEFT JOIN planes ON flights.tailnum=planes.tailnum
""", conn)

# Pandas Equivalent 
task15_pandas = flights.merge(planes[['tailnum', 'year', 'speed', 'seats']],
                              how='left', left_on='tailnum', right_on='tailnum')

# Rename columns to match the SQL result
task15_pandas.columns = ['year', 'month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay',
                         'arr_time', 'sched_arr_time', 'arr_delay', 'carrier', 'flight',
                         'tailnum', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute',
                         'time_hour', 'plane_year', 'plane_speed', 'plane_seats']



# Compare the results
pd.testing.assert_frame_equal(task15_sql, task15_pandas)

Task 15 SQL query performs a LEFT JOIN between the flights and planes tables on the 'tailnum' column, including additional columns from the planes table. The Pandas equivalent achieves this using the merge function without pd.merge, matching on 'tailnum'. Column names are adjusted to align with the SQL result. The Pandas and SQL results are printed, and assert_frame_equal ensures their equality. The code leverages merge for a concise alternative to pd.merge, improving code readability and maintaining consistency with SQL logic.

In [ ]:
# SQL Query for Task 16
task16_sql = pd.read_sql_query('''
    SELECT planes.*, airlines.*
    FROM (
        SELECT DISTINCT carrier, tailnum
        FROM flights
    ) AS cartail
    INNER JOIN planes ON cartail.tailnum = planes.tailnum
    INNER JOIN airlines ON cartail.carrier = airlines.carrier
''', conn)

# Creating DataFrame cartail
cartail = flights[['carrier', 'tailnum']].drop_duplicates()

# Inner join with planes DataFrame
task16_my = pd.merge(cartail, planes, on='tailnum', how='inner')

# Inner join with airlines DataFrame
task16_my = pd.merge(task16_my, airlines, on='carrier', how='inner')

# Explicitly order the columns in both DataFrames
task16_my = task16_my[['tailnum', 'year', 'type', 'manufacturer', 'model', 'engines', 'seats', 'speed', 'engine', 'carrier', 'name']]

# Sorting the DataFrame to ensure order consistency
task16_my.sort_values(by=['tailnum', 'carrier'], inplace=True)

# Resetting the index
task16_my.reset_index(drop=True, inplace=True)

# To check if SQL and Pandas results are equal
pd.testing.assert_frame_equal(task16_sql, task16_my)

Task 16 SQL query selects unique combinations of carrier and tailnum from the flights table and performs INNER JOINs with the planes and airlines tables. The Pandas equivalent achieves this by using drop_duplicates on the relevant columns, then merging with planes and airlines separately. The results are printed for both SQL and Pandas, and assert_frame_equal ensures their equality. This approach ensures matching results in both SQL and Pandas by explicitly defining the intermediate steps and combining them, maintaining the logic of the SQL query in the Pandas code.

Post Graduate Part

In [ ]:
 #Equivalent SQL Query
task17_sql = pd.read_sql_query("""
    SELECT flights2.*, atemp, ahumid
    FROM (
        SELECT * FROM flights WHERE origin='EWR'
    ) AS flights2
    LEFT JOIN (
        SELECT
            year, month, day,
            AVG(temp) AS atemp,
            AVG(humid) AS ahumid
        FROM weather
        WHERE origin='EWR'
        GROUP BY year, month, day
    ) AS weather2
    ON flights2.year=weather2.year
    AND flights2.month=weather2.month
    AND flights2.day=weather2.day;
""", conn)

# Pandas Equivalent
flights2 = flights[flights['origin'] == 'EWR'].copy()
weather2 = weather[weather['origin'] == 'EWR'].groupby(['year', 'month', 'day']).agg(atemp=('temp', 'mean'), ahumid=('humid', 'mean')).reset_index()

task17_pandas = pd.merge(flights2, weather2, on=['year', 'month', 'day'], how='left')



pd.testing.assert_frame_equal(task17_sql, task17_pandas)

For Task 17, the Pandas code selects flights and weather data for the 'EWR' origin. It calculates daily average temperature ('atemp') and humidity ('ahumid') from the weather data, grouping by year, month, and day. A left join is performed on flights' date-related columns, replicating the SQL logic. The results are printed for both SQL and Pandas, and assert_frame_equal is employed for validation, considering ordering and data types for consistency.


# **Conclusion:**
In this analysis, data from the 'nycflights13' dataset was processed using both SQL queries and Pandas operations in Python. The tasks covered a range of operations, including selecting unique values, filtering based on conditions, aggregating data, and performing joins.

Tasks 1 to 5 involved basic queries, such as selecting distinct engine types and type-engine pairs, counting occurrences of engine types, and finding combinations of engine types and plane types with their counts and order statistics. Tasks 6 to 10 focused on more complex queries, including aggregating and summarizing data based on conditions, such as finding minimum, average, and maximum years for each combination of engine and manufacturer.

Tasks 11 to 17 introduced additional complexity with filtering and joining operations. The queries involved selecting manufacturers with more than 200 seats and ordering the results. Task 13 included a 'HAVING' clause in both SQL and Pandas, emphasizing a condition on the grouped data. Tasks 15 and 16 demonstrated joining tables to retrieve specific columns from related tables.

Throughout the tasks, the Pandas code closely mirrored the SQL queries, ensuring equivalent results. The 'assert_frame_equal' method was employed to validate the consistency of results, considering both data values and column order.

Overall, the Pandas library provides a powerful and flexible alternative to SQL for data manipulation and analysis, offering a seamless transition for those familiar with SQL queries. The combination of SQL and Pandas allows for comprehensive data exploration and analysis, providing insights into the 'nycflights13' dataset.